Recommender systems
================

Qué es?
=======

![](./images/mask.png)

- Sistema capaz de predecir el **rating** que un usuario dará a elementos de un conjunto
- Usualmente basado en tener ratings anteriores de esa persona y de otros usuarios
- Una de las aplicaciones más importantes de machine learning
- Muy populares en los últimos años

### Aplicaciones reales

- Amazon / Mercado libre ...
- Netflix / Youtube / Spotify ...
- Facebook / Twitter / ...
- Google
- Banca/seguros
- ...

| Película         | Pedro | Emilia | Martín  | Paola  |
|------------------|:-----:| ------:|--------:|-------:|
| Plaga zombie     |     ? |      ? |      ?  |     ?  |
| Freddy vs Jason  |     ? |      ? |      ?  |     ?  |
| Saw 1            |     ? |      ? |      ?  |     ?  |
| Sintonía de amor |     ? |      ? |      ?  |     ?  |
| Titanic          |     ? |      ? |      ?  |     ?  |?


$n_u$ = cantidad de usuarios/personas

$n_m$ = cantidad de películas

$r(i,j)$ = 1 si el usuario **"j"** dejó su opinión sobre la película **"i"**, 0 si no

$y(i,j)$ = rating del usuario **"j"** sobre la película **"i"**

En base a tener algunos de esos datos, vamos a intentar predecir los que no tenemos.

### Feedback: explícito o implícito?

- Muchas veces tenemos el feedback directamente: stars, +1, "me gusta", ...
- Pero ese datos muchas veces no está presente.
- A veces se puede deducir!

Generalmente se dispone de algún tipo de feedback, explícito o implícito: si el usuario vio un video completo o no, cantidad de veces que puso pausa o que volvió hacia atrás, número de veces que vio o accedió a un video, etc.

## Recomendaciones basadas en contenido

Se basa en la idea de que dos elementos parecidos entre sí deberían recibir opiniones similares por parte de un mismo usuario.

**No** conocemos el detalle de las preferencias de los usuarios (no se si a Pedro le gustan las de terror o las románticas).

**Sí** conocemos las características de cada película (si es de terror o romántica).

**Sí** conocemos algunas puntuaciones de los usuarios a algunas películas.

"Pedro no me dijo qué género le gusta, pero estas dos películas son de terror y dijo que le gustaron, así que le deben gustar las películas de terror".

| Película        | Pedro | Emilia | Martín  | Paola | X1 (terror) |  X2 (romántica) |
| ----------------|:-----:| ------:|--------:|-------|-------------|-----------------|
| Plaga zombie    |   5   |    5   |    0    |   0   |     0.9     |       0         |
| Freddy vs Jason |   5   |    ?   |    ?    |   0   |     1.0     |       0.01      |            
| Saw 1           |   ?   |    4   |    0    |   ?   |     0.99    |       0         |
| Sintonía de amor|   0   |    0   |    5    |   4   |     0.1     |       1.0       |
| Titanic         |   0   |    ?   |    5    |   ?   |     0       |       0.9       |

Por cada usuario voy a intentar aprender los parámetros de la recta que intentan replicar mejor los valores que tengo. Ejemplo:

$f_{pedro}(x) = x_1 * 3.7 + x_2 * 0.2 + 1$

3.7, 0.2 y 1 son los valores que tengo que encontrar, los **parámetros** para Pedro ($\theta_{pedro} = [3.7, 0.2, 1]$).

La función recibe una película (como features $x$), y devuelve el rating que Pedro le da a esa película. La película es un vector de valores que la describen (en este ejemplo, "nivel de cada género").

Vamos a expresar esa función de recta en forma vectorial, como siempre:

$f_{pedro}(x) = x_1 * 3.7 + x_2 * 0.2 + 1$

se convierte en:

$f_{pedro}(x) = (\theta_{pedro})^T x$

**¿Cómo encontramos Θ?** ¿Empezamos a probar al azar ?

![](images/error_function.svg)

Podemos tratar de armar una función de error. Cuál es la diferencia entre una predicción y lo que realmente sabemos?: 

$f_{pedro}(x) - y^{(i,pedro)} $

O lo que es lo mismo:

$(\theta_{pedro})^T x - y^{(i,pedro)} $


"el raiting que predecimos de una película, cuánto difiere del rating que conocemos que Pedro le dio?"

Generalizada para el set entero de películas:

$E_{pedro} = \sum_{i:r(i,pedro)=1} ((\theta_{pedro})^T X^{(i)} - y^{(i,pedro)})^2$

"para todas las películas (i) donde tenemos el raiting de Pedro, cuánto difiere nuestro rating predicho del rating conocido?" (error cuadrático)

Y con la función de error, entonces volvemos a nuestro viejo conocido...

## Descenso por el gradiente

![](images/gradient_descent.svg)

Lo que terminamos aprendiendo son entonces **las preferencias de Pedro**. Cuánto le gustan las de terror, cuánto las románticas. Esos son sus pesos.

### Y el resto de los usuarios?

Tendría que armar una función por cada usuario? Tenemos que **entrenar un modelo por cada usuario**?? Por ahora parece que sí.

Podemos en vez redefinir la función de error e integrar las salidas para todos los usuarios, para tener **una sola función global, que sirve para todos los usuarios**:

$E_{total} = \sum_{j=1}^{n_u} \sum_{i:r(i,j)=1} ((\theta^{(j)})^T X^{(i)} - y^{(i,j)})^2$

"para todos los usuarios (j), para todas las películas (i), donde tengamos el raiting del usuario para esa película, cuánto difiere nuestro rating predicho del rating conocido?" (error cuadrático)

Ahora θ, lo que aprendemos, es un array de dos dimensiones, con los valores para cada usuario:

| Usuario |  θ(0)  |  θ(1)  |  θ(2) |
| --------| ------:|-------:|-------|
| Pedro   |    3.7 |    0.2 |   1.0 |
| Emilia  |    4.1 |    0.5 |   0.3 |
| Martín  |    0.1 |    4.3 |   0.1 |
| Paola   |    1.2 |    5.3 |   1.4 |


Con θ, ahora podemos agarrar a cualquier usuario y cualquier película (de las que ya conocemos las características por definición), y usar la función para predecir qué tanto le va a gustar esa película.

Con eso podemos hacer mil cosas (rankear recomendaciones, sugerir con qué seguir, ordenar búsquedas, etc).

## Recomendaciones basadas en filtros colaborativos

Se basa en la idea de que dos personas que tuvieron opiniones parecidas en el pasado también las tendrán en el futuro.

**No** conocemos explicitamente el detalle de las preferencias de los usuarios.

**No** conocemos las características de cada película (si es de terror o romántica, o de qué género).

**Sí** conocemos algunas puntuaciones de los usuarios a algunas películas.

"Pedro no me dijo qué género le gusta y encima tampoco se de qué género es cada película, pero a él y a Emilia le suelen gustar las mismas películas. Entonces si esta película le gustó a Emilia, probablemente le guste también a Pedro".

Ej: Tiktok, Instagram, Youtube, etc.

| Película        | Pedro | Emilia | Martín  | Paola | X1 (terror) |  X2 (romántica) |
| ----------------|:-----:| ------:|--------:|-------|-------------|-----------------|
| Plaga zombie    |   5   |    5   |    0    |   0   |     ?       |       ?         |
| Freddy vs Jason |   5   |    ?   |    ?    |   0   |     ?       |       ?         |            
| Saw 1           |   ?   |    4   |    0    |   ?   |     ?       |       ?         |
| Sintonía de amor|   0   |    0   |    5    |   4   |     ?       |       ?         |
| Titanic         |   0   |    0   |    5    |   ?   |     ?       |       ?         |

#### Qué pasaría si las personas nos dijeran sus preferencias?


Si tenemos todos los vectores de parámetros ...

$\theta(pedro) = [5, 0, 0]$

$\theta(emilia) = [4, 1, 0]$

$\theta(martin) = [0, 5, 0]$

$\theta(paola) = [1, 4, 0]$

Pero no tenemos los vectores de las películas... cómo podría ser la función de error?

Podría ser igual!

$E_{total} = \sum_{j=1}^{n_u} \sum_{i:r(i,j)=1} ((\theta^{(j)})^T X^{(i)} - y^{(i,j)})^2$

"para todos los usuarios (j), para todas las películas (i), donde tengamos el raiting del usuario para esa película, cuánto difiere nuestro rating predicho del rating conocido?" (error cuadrático)

**Solo tenemos un cambio**: antes $\theta$ eran los pesos que no conocíamos y teníamos que aprender. Ahora esos ya los sabemos, están fijos, y lo que no conocemos y tenemos que aprender es...

$X!$

(los vectores que describen a cada película)

#### Qué pasaría si las personas NO nos dijeran sus preferencias?


Podríamos utilizar los y(i, j) conocidos para aprender tanto las preferencias de los usuarios ($\theta$) como los vectores de las películas ($X$) al mismo tiempo. Misma función de error, y ahora tanto $\theta$ como $X$ son parámetros a aprender!

$E_{total} = \sum_{j=1}^{n_u} \sum_{i:r(i,j)=1} ((\theta^{(j)})^T X^{(i)} - y^{(i,j)})^2$

Una vez que eso está entrenado, tenemos tanto preferencias de los usuarios como características del contenido, podemos hacer predicciones con nuestra función de siempre: $(\theta^{(pedro)})^T X^{(peli)}$

Qué significan ahora los valores de $\theta$ y $X$?

De qué género es una película que terminó con $X^{(peli)} = [4.9, 0.1, 0]$ ? 

Qué películas le gustan a un usuario que terminó con $\theta^{(usuario)} = [4.5, 0.2, 0.5]$ ?

No sabemos! Las preferencias y características ya no son definidas por nosotros! Solo elegimos "entrename 2 características" (y el bias), y el modelo encontró 2 valores para usar ahí, que hacen que las predicciones anden bien.

## Problemas comunes

- Cold start: nuevo usuario? nueva pelicula?
- Escalabilidad: En muchos casos podemos tener cientos de miles o millones de cosas para recomendar, lo cual hace que obtener recomendaciones de forma automática y al instante sea todo un desafío.
- Datos dispersos (sparse data): Generalmente tenemos data de una número mucho menor al total disponible de elementos.

## Actualidad: enfoque híbridos

Existe evidencia empírica de que combinar ambos modelos puede ser una buena alternativa para obtener resultados mejores que si se utilizaran de forma aislada.

Existen distintas principalmente dos formas de combinarlos:
- Combinando las salidas de ambos.
- Creando un único modelo que combine ambos enfoques (ej: algunas características cargadas por humanos y otras aprendidas).

Un caso de ejemplo es Netflix, que realiza recomendaciones de películas a partir del historial de búsqueda y visitas de otros usuarios (filtros colaborativos) y de películas que tienen características similares (basados en contenido).

## Métricas

Definir medidas de performance varía en relación a lo que se hace para un problema de clasificación común y corriente.

¿Que hagamos una recomendación y que el usuario no la haya seleccionado, significa que estaba mal?

Para medición offline se suelen usar metricas de ranking, entre ellas: 
* Precision at *n* (P@n) ("de los primeros N contenidos que recomendamos, cuáles realmente están en el top N de lo que le gusta al usuario?")
* MAP (mean average precision) ("en promedio, qué tanto se parecen nuestras N recomendaciones para cada usuario, a las top N cosas que sabemos que le gustan?")

## Y no solo "qué tanto acertamos".

Surgen otro tipo de métricas importantes:

- Diversidad: mide cuánto se parecen las recomendaciones entre si (la idea subyacente es que a mayor diversidad, más probabilidades de acertar).
- Persistencia: mide el grado en el que ante situaciones semejantes, el criterio de selección se mantiene consistente.
- Privacidad: generalmente existen restricciones legales sobre la privacidad de los datos de los usuarios, lo cual limitan o condicionan la utilidad de un determinado sistema.
- Serendipity: Mide el grado en que una recomendación es predecible o "esperable".
- Coverage: a cuantos usuarios les puedo hacer recomendación